# Step 1: Activate Virtual Environment & Run the MCP Server
```Run Command (From project-root directory):  uv run .\mcp_server\main.py```

# Step 2: Connect Server to MCP Client & Grab Tools

In [1]:
import json
from mcp_client import McpClientPool, SEPARATOR # The separator is used for avoiding duplicate tool names. 

# NOTE: Since each server requires it's own MCP client, 
# we can manage multiple clients with an MCP client pool
# that holds all the connections. This all|ws us to 
# safely open and close all clients.
mcp_pool = McpClientPool()

# Setting up the server information 
# NOTE: Local Servers!
#   - Local servers will always have the /mcp
#   - By default, all local servers will be HTTP not HTTPS
server_id = "demo"
server_url = "http://127.0.0.1:4000/mcp" 

# Connecting to the server and saving the tools
await mcp_pool.add_client(name=server_id, base_url=server_url)

# NOTE: All the tools available also exist in the mcp_pool's instance field all_tools (mcp.all_tools).
# However, for demo purposes we will explicitly get the tools
tools = await mcp_pool.list_tools(server_id)

In [ ]:
for i, t in enumerate(tools):
    print(f"Tool {i+1}: ")
    print(json.dumps(t, indent=4), end="\n\n")

# Step 3: Ask a question

In [2]:
query = "What is the MCP, Model Context Protocol?"

# Step 4: Pass Query and Tools to LLM & Select Best Tool

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

openai_client = AzureOpenAI(
    api_key=os.getenv('AZURE_API_KEY'),
    api_version=os.getenv('VERSION'),
    azure_endpoint=os.getenv('ENDPOINT'),
)

response = openai_client.chat.completions.create(
    model=os.getenv('MODEL'),  
    messages=[
        # Since GPT's internal datasource is huge, sometimes the model will directly answer the query. 
        # For demo purposes we force it to select a tool via the prompt.
        {"role": "user", "content": f"You must select a tool from the provided tools given a query {query}"}
    ],
    functions=tools,
    function_call="auto"  # "auto" lets the model decide to call the tool
)

selected_tool = response.choices[0].message.function_call
print(f"Selected Tool: {selected_tool.name}")
print(f"Arguments: {selected_tool.arguments}")

Selected Tool: demo__search_internet_and_answer
Arguments: {"query":"What is the MCP, Model Context Protocol?"}


# Step 5: Call The Tool!

In [5]:
from IPython.display import display, Markdown
# Splitting the selected tool to obtain the server and the tool's name
server_name, tool_name = selected_tool.name.split(SEPARATOR)
# Loading the arguments as a JSON (str -> JSON)
loaded_args = json.loads(selected_tool.arguments)

# Calling the tool on our MCP Server
# NOTE: method is NOT the name of the tool, 
# it's the name of the route on the server
tool_response = await mcp_pool.call(
    name=server_name,
    method="tools/call",
    params={
        "name": tool_name,
        "arguments": loaded_args
    }
)

answer = tool_response[0]['result']['content'][0]['text']
display(Markdown(answer))

### What’s the MCP (Model Context Protocol) in plain English?

The **MCP (Model Context Protocol)** is basically like a universal plug for connecting AI assistants (like Claude) to wherever your data is stored—Google Drive, Slack, GitHub, databases, you name it.

#### Why does this even matter?
AI models are often **stuck in info jail**. They’re smart, but can’t magically grab data from random places unless someone builds a custom bridge for each one. Every time you want your AI to access a new tool or database, developers have to create a custom “connector”—which is a pain, slow, and not scalable. 🤦

#### Enter MCP. 🚀
MCP is an **open standard** (think: everyone can use it, contribute, and build cool stuff). Instead of building a million one-off connectors, everyone uses **one protocol** to link AI models to different data sources. It’s kind of like how USB made it easy to connect all kinds of devices to your laptop—MCP does the same but for AI and your data.

#### How does it work?  
- **Developers** can expose their data using MCP servers (these act like the “middleman”).
- **AI apps** connect to these servers through MCP clients.
- Anthropic is making it easy by sharing pre-made MCP servers for common platforms (Google Drive, Slack, GitHub, etc.), so you don’t have to start from scratch.

#### Real world vibes:
- **Block & Apollo** (big companies) are already using this to make their AI tools smarter and more connected.
- **Dev tool companies** (Zed, Replit, Sourcegraph, etc.) integrate MCP so their AI agents can grab more context for coding tasks, making less “dumb” mistakes and writing better code.

#### TL;DR:
MCP is an open-source “universal language” for AI models to talk to any kind of data source, making your AI way more useful and less of a socially awkward robot. 🔌🤖

---

**Want to build with it?** You can with the Claude Desktop app, or dive into Anthropic’s open-source repos to make your own connectors! It’s collaborative, open to everyone, and built for a more connected future.

# Step 6: Close the Pool When Finished

In [6]:
await mcp_pool.close_all()